In [1]:
using RigidBodyTreeInspector
using RigidBodyDynamics
import DataStructures: OrderedDict
using GeometryTypes
using StaticArrays

adding: /home/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/v0.5/DrakeVisualizer/src/lcmtypes to the python path


In [2]:
import Flash
reload("Flash")

In [3]:
urdf = "/home/rdeits/locomotion/explorations/point-cloud-signed-distance/examples/data/IRB140/urdf/irb_140_convhull.urdf"
mechanism = parse_urdf(Float64, urdf);

In [4]:
vis = parse_urdf(urdf, mechanism);
inspect(mechanism, vis)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"joint1.1",0.0,linspace(-3.141592653589793,3.141592653589793,51),".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"joint2.1",0.0,linspace(-3.141592653589793,3.141592653589793,51),".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"joint3.1",0.0,linspace(-3.141592653589793,3.141592653589793,51),".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"joint4.1",0.0,linspace(-3.141592653589793,3.141592653589793,51),".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"joint5.1",0.0,linspace(-3.141592653589793,3.141592653589793,51),".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"joint6.1",0.0,linspace(-3.141592653589793,3.141592653589793,51),".3f",true)

nothing

In [5]:
vis_data = RigidBodyTreeInspector.parse_urdf_visuals(urdf, mechanism);

In [6]:
surfaces = Vector{Flash.Surface{Float64}}()
for (i, node) in enumerate(mechanism.toposortedTree)
    body = node.vertexData
    mesh = vis_data[i].geometry_data[1].geometry
    tform = vis_data[i].geometry_data[1].transform
    verts = vertices(mesh)
    surface_points = [Point3D(body.frame, tform(SVector{3, Float64}(v...))) for v in verts]
    skeleton_points = Vector{Point3D{Float64}}()
    geometries = OrderedDict(body => Flash.BodyGeometry(surface_points, skeleton_points))
    push!(surfaces, Flash.Surface(geometries, Flash.RigidPolytope()))
end

model = Flash.Manipulator(mechanism, surfaces);
state = Flash.ManipulatorState(model);
skin = Flash.skin(state);

In [7]:
Visualizer(DrakeVisualizer.contour_mesh(skin, Vec(-.5, -.5, -.25), Vec(1., .5, 1), 0.01, 0.05))

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 2242xFixedSizeArrays.Point{3,Float64},     faces: 4480xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"link")]),1,PyLCM.LCM(PyObject <LCM object at 0x7f1f2d9e6570>))